# Skin lesion Classification and segementation using the pretrianed model RESNET 50 

Importing the library

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import layers, models, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from sklearn.metrics import confusion_matrix, f1_score



In [2]:
train_image_dir = r"C:\Users\Administrator\Desktop\Abhishek\train"
train_metadata_path = r"C:\Users\Administrator\Desktop\Abhishek\Skin-Lesion-Classification-and-Segmentation\train_metadata.csv"
val_image_dir = r"C:\Users\Administrator\Desktop\Abhishek\val"
val_metadata_path = r"C:\Users\Administrator\Desktop\Abhishek\Skin-Lesion-Classification-and-Segmentation\val_metadata.csv"
test_image_dir = r"C:\Users\Administrator\Desktop\Abhishek\test"
test_metadata_path = r"C:\Users\Administrator\Desktop\Abhishek\Skin-Lesion-Classification-and-Segmentation\test_metadata.csv"

In [3]:

def load_image_label(image_path, dx):
    try:
        image_data = tf.io.read_file(image_path)
        image_data = tf.image.decode_jpeg(image_data, channels=3)  # Load image
        image_data = tf.image.resize(image_data, [256, 256])       # Ensure it is 256x256
        image_data = tf.image.convert_image_dtype(image_data, tf.float32)  # Normalize to [0, 1]
        dx = tf.cast(dx, tf.int32)
        return image_data, dx
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return tf.zeros([256, 256, 3], dtype=tf.float32), tf.constant(-1, dtype=tf.int32)


In [4]:
def create_dataset(image_dir, metadata_path, batch_size=32, shuffle=True, sample_count=None):
    df = pd.read_csv(metadata_path)

    if sample_count is not None:
        df = df.iloc[:sample_count]

    # Automatically create label mapping
    if df['dx'].dtype == object:
        unique_labels = sorted(df['dx'].unique())
        label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
        df['dx'] = df['dx'].map(label_mapping)

    image_paths = df['image_id'].apply(lambda x: os.path.join(image_dir, x + ".jpg")).tolist()
    labels = df['dx'].tolist()
    ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    ds = ds.map(lambda path, dx: load_image_label(path, dx),
                num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(buffer_size=sample_count or len(df), seed=42)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds


In [5]:

batch_size = 32
train_ds = create_dataset(train_image_dir, train_metadata_path, batch_size=batch_size, shuffle=True, sample_count=12000)
val_ds   = create_dataset(val_image_dir, val_metadata_path, batch_size=batch_size, shuffle=False, sample_count=6000)
test_ds  = create_dataset(test_image_dir, test_metadata_path, batch_size=batch_size, shuffle=False, sample_count=5000)


In [6]:

def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    # Reduced brightness and contrast augmentation
    image = tf.image.random_brightness(image, max_delta=0.05)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    # Limit rotation to 0° or 90° (instead of 0°, 90°, 180°, 270°)
    k = tf.random.uniform(shape=[], minval=0, maxval=2, dtype=tf.int32)
    image = tf.image.rot90(image, k)
    return image, label

train_ds = train_ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE)


In [7]:
# Debug: Check a batch's shapes
for img_batch, dx_batch in train_ds.take(1):
    print(f"Image batch shape: {img_batch.shape}")   # Expected: (32, 256, 256, 3)
    print(f"DX batch shape: {dx_batch.shape}")         # Expected: (32,)


NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} NewRandomAccessFile failed to Create/Open: C:\Users\Administrator\Desktop\Abhishek\train\ISIC_0028880_aug3014.jpg : The system cannot find the file specified.
; No such file or directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext] name: 

Load the pretrained model

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

num_classes = 7  # Update based on your dataset

# Load pre-trained MobileNetV2 (without the top classification layer)
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(256, 256, 3)
)

# Freeze base layers
base_model.trainable = False

# Define the model function
def mobilenet(input_shape, num_classes):
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')  # ✅ Using num_classes directly
    ])
    return model

# Create and summarize the model
input_shape = (256, 256, 3)
mobilenet_model = mobilenet(input_shape, num_classes)
mobilenet_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 8, 8, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 7)                 1799      
                                                                 
Total params: 2587719 (9.87 MB)
Trainable params: 329735

In [ ]:
import os
import re
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import load_model

# === 1. Settings ===
checkpoint_dir = './checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_pattern = os.path.join(checkpoint_dir, "mob-{epoch:02d}.keras")

# === 2. Get latest checkpoint (if any) ===
def get_latest_checkpoint():
    files = [f for f in os.listdir(checkpoint_dir) if f.startswith("mob") and f.endswith(".keras")]
    if not files:
        return None, 0
    def extract_epoch(fname):
        match = re.search(r'mob-(\d+)\.keras', fname)
        return int(match.group(1)) if match else -1
    latest_file = max(files, key=extract_epoch)
    latest_epoch = extract_epoch(latest_file)
    return os.path.join(checkpoint_dir, latest_file), latest_epoch

latest_checkpoint_path, latest_epoch = get_latest_checkpoint()

# === 3. Load or build model ===
if latest_checkpoint_path:
    print(f"🔁 Resuming from checkpoint: {latest_checkpoint_path}")
    mobilenet_model = load_model(latest_checkpoint_path)
else:
    print("🆕 Starting fresh training...")
    mobilenet_model = mobilenet(input_shape, num_classes)  # You must define this function

# === 4. Custom callback to save every 15 epochs ===
class SaveEveryNEpochs(Callback):
    def __init__(self, save_freq, checkpoint_template):
        super().__init__()
        self.save_freq = save_freq
        self.checkpoint_template = checkpoint_template

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            filepath = self.checkpoint_template.format(epoch=epoch + 1)
            self.model.save(filepath)
            print(f"✅ Saved full model at {filepath}")

checkpoint_callback = SaveEveryNEpochs(save_freq=15, checkpoint_template=checkpoint_pattern)




🔁 Resuming from checkpoint: ./checkpoints\mob-02.keras


In [ ]:

mobilenet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy", SparseCategoricalAccuracy()]
)

In [ ]:

early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# ---------------------------------------
# Learning Rate Scheduler callback to reduce LR when validation loss plateaus
# ---------------------------------------
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-06)


In [ ]:

EPOCHS = 150
history = mobilenet_model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=EPOCHS, 
    initial_epoch=latest_epoch,
    callbacks=[early_stopping, reduce_lr,checkpoint_callback], 
    verbose=1
)

Epoch 3/150
 11/375 [..............................] - ETA: 2:35 - loss: 1.5582 - accuracy: 0.3608 - sparse_categorical_accuracy: 0.3608

KeyboardInterrupt: 